In [1]:
import shutil
import os
import tensorflow as tf
import numpy as np
from sklearn.utils import class_weight
from tensorflow.keras.applications.mobilenet_v2 import preprocess_input
from tensorflow.keras import backend as K

In [2]:
# Clear previous TF session
K.clear_session()

In [3]:
# ============================
# Parameters
# ============================
data_dir = "data"
img_size = (160, 160)
batch_size = 32
epochs_top = 10
epochs_fine = 5
AUTOTUNE = tf.data.AUTOTUNE
cache_train_path = "./train_cache.tf-data"
cache_val_path = "./val_cache.tf-data"

In [4]:

# ============================
# Clear cache directories if they exist
# ============================
for cache_path in [cache_train_path, cache_val_path]:
    if os.path.exists(cache_path):
        shutil.rmtree(cache_path)  # delete cache folder
        print(f"Cleared cache: {cache_path}")

In [5]:
# ============================
# Load Dataset
# ============================
train_ds = tf.keras.utils.image_dataset_from_directory(
    data_dir,
    validation_split=0.2,
    subset="training",
    seed=123,
    image_size=img_size,
    batch_size=batch_size
)

val_ds = tf.keras.utils.image_dataset_from_directory(
    data_dir,
    validation_split=0.2,
    subset="validation",
    seed=123,
    image_size=img_size,
    batch_size=batch_size
)


Found 114022 files belonging to 15 classes.
Using 91218 files for training.
Found 114022 files belonging to 15 classes.
Using 22804 files for validation.


In [6]:
class_names = train_ds.class_names
num_classes = len(class_names)
print("Classes:", class_names)

Classes: ['Acne And Rosacea Photos', 'Actinic Keratosis Basal Cell Carcinoma And Other Malignant Lesions', 'Atopic Dermatitis Photos', 'Ba  Cellulitis', 'Ba Impetigo', 'Benign', 'Bullous Disease Photos', 'Cellulitis Impetigo And Other Bacterial Infections', 'Eczema Photos', 'Exanthems And Drug Eruptions', 'Fu Athlete Foot', 'Fu Nail Fungus', 'Fu Ringworm', 'Hair Loss Photos Alopecia And Other Hair Diseases', 'Heathy']


In [7]:
# ============================
# Data Augmentation
# ============================
data_augmentation = tf.keras.Sequential([
    tf.keras.layers.RandomFlip("horizontal"),
    tf.keras.layers.RandomRotation(0.2),
    tf.keras.layers.RandomZoom(0.2),
    tf.keras.layers.RandomContrast(0.15),
])


In [8]:
# ============================
# Preprocessing
# ============================
def preprocess_train(x, y):
    x = tf.image.resize(x, img_size)
    x = data_augmentation(x, training=True)
    x = preprocess_input(x)
    return x, y

def preprocess_val(x, y):
    x = tf.image.resize(x, img_size)
    x = preprocess_input(x)
    return x, y

train_ds = train_ds.map(preprocess_train, num_parallel_calls=AUTOTUNE)
val_ds = val_ds.map(preprocess_val, num_parallel_calls=AUTOTUNE)


In [9]:
# ============================
# Cache datasets (in-memory, auto-clear on program exit)
# ============================
train_ds = train_ds.cache().prefetch(AUTOTUNE)
val_ds = val_ds.cache().prefetch(AUTOTUNE)


In [10]:
# ============================
# Class Weights
# ============================
y_train = np.concatenate([y for x, y in train_ds], axis=0)
class_weights = dict(enumerate(class_weight.compute_class_weight(
    'balanced', classes=np.unique(y_train), y=y_train
)))
print("Class weights:", class_weights)

: 

In [ ]:
# ============================
# Model (MobileNetV2)
# ============================
base_model = tf.keras.applications.MobileNetV2(
    input_shape=img_size + (3,),
    include_top=False,
    weights='imagenet'
)
base_model.trainable = False

x = tf.keras.layers.GlobalAveragePooling2D()(base_model.output)
output = tf.keras.layers.Dense(num_classes, activation='softmax', dtype='float32')(x)

model = tf.keras.Model(inputs=base_model.input, outputs=output)

In [ ]:
# ============================
# Compile & Train Top Layers
# ============================
model.compile(
    optimizer=tf.keras.optimizers.Adam(1e-3),
    loss='sparse_categorical_crossentropy',
    metrics=['accuracy']
)
history_top = model.fit(
    train_ds,
    validation_data=val_ds,
    epochs=epochs_top,
    class_weight=class_weights
)

In [ ]:

# ============================
# Fine-Tuning Last 50 Layers
# ============================
base_model.trainable = True
for layer in base_model.layers[:-50]:
    layer.trainable = False

model.compile(
    optimizer=tf.keras.optimizers.Adam(1e-5),
    loss='sparse_categorical_crossentropy',
    metrics=['accuracy']
)
history_fine = model.fit(
    train_ds,
    validation_data=val_ds,
    epochs=epochs_fine,
    class_weight=class_weights
)

In [ ]:



# ============================
# Save Model
# ============================
save_path = "./tf_model"
model.save(save_path)
print(f"Model saved to {save_path} ✅")


In [ ]:
# ============================
# 0. Imports & Clear Session
# ============================
import tensorflow as tf
import numpy as np
from sklearn.utils import class_weight
from tensorflow.keras.applications import MobileNetV3Small
from tensorflow.keras.applications.mobilenet_v3 import preprocess_input
from tensorflow.keras import backend as K
from tensorflow.keras.callbacks import EarlyStopping

# Clear previous TF session
K.clear_session()
print("✅ TensorFlow session cleared.")

# ============================
# 1. Paths & Parameters
# ============================
data_dir = "data"
img_size = (128, 128)
batch_size = 32
epochs_top = 20
AUTOTUNE = tf.data.AUTOTUNE

print(f"📁 Dataset path: {data_dir}")
print(f"🖼 Image size: {img_size}, Batch size: {batch_size}")

# ============================
# 2. Load Dataset
# ============================
train_ds = tf.keras.utils.image_dataset_from_directory(
    data_dir,
    validation_split=0.2,
    subset="training",
    seed=123,
    image_size=img_size,
    batch_size=batch_size
)
val_ds = tf.keras.utils.image_dataset_from_directory(
    data_dir,
    validation_split=0.2,
    subset="validation",
    seed=123,
    image_size=img_size,
    batch_size=batch_size
)

class_names = train_ds.class_names
num_classes = len(class_names)
print(f"📌 Classes detected ({num_classes}): {class_names}")

# ============================
# 3. Data Augmentation
# ============================
data_augmentation = tf.keras.Sequential([
    tf.keras.layers.RandomFlip("horizontal"),
    tf.keras.layers.RandomRotation(0.2),
    tf.keras.layers.RandomZoom(0.2),
    tf.keras.layers.RandomContrast(0.15),
])
print("🎨 Data augmentation pipeline created.")

# ============================
# 4. Preprocessing
# ============================
def preprocess_train(x, y):
    x = tf.image.resize(x, img_size)
    x = data_augmentation(x, training=True)
    x = preprocess_input(x)
    return x, y

def preprocess_val(x, y):
    x = tf.image.resize(x, img_size)
    x = preprocess_input(x)
    return x, y

train_ds = train_ds.map(preprocess_train, num_parallel_calls=AUTOTUNE)
train_ds = train_ds.cache().prefetch(AUTOTUNE)
print("✅ Training dataset preprocessed and cached in memory.")

val_ds = val_ds.map(preprocess_val, num_parallel_calls=AUTOTUNE)
val_ds = val_ds.cache().prefetch(AUTOTUNE)
print("✅ Validation dataset preprocessed and cached in memory.")

# ============================
# 5. Class Weights
# ============================
y_train = np.concatenate([y for x, y in train_ds], axis=0)
class_weights = dict(enumerate(class_weight.compute_class_weight(
    'balanced', classes=np.unique(y_train), y=y_train
)))
print(f"⚖️ Class weights calculated: {class_weights}")

# ============================
# 6. Build Model (MobileNetV3 Small)
# ============================
base_model = MobileNetV3Small(
    input_shape=img_size + (3,),
    include_top=False,
    weights='imagenet'
)
base_model.trainable = False

x = tf.keras.layers.GlobalAveragePooling2D()(base_model.output)
output = tf.keras.layers.Dense(num_classes, activation='softmax', dtype='float32')(x)
model = tf.keras.Model(inputs=base_model.input, outputs=output)
print("🧱 MobileNetV3 Small model created (feature extraction).")

# ============================
# 7. Compile Model
# ============================
model.compile(
    optimizer=tf.keras.optimizers.Adam(learning_rate=0.001),
    loss='sparse_categorical_crossentropy',
    metrics=['accuracy']
)
print("⚙️ Model compiled.")

# ============================
# 8. Early Stopping Callback
# ============================
early_stop = EarlyStopping(
    monitor='val_accuracy',
    patience=3,
    restore_best_weights=True
)
print("⏹ EarlyStopping callback created.")

# ============================
# 9. Train Model
# ============================
print("🚀 Starting training (feature extraction)...")
history = model.fit(
    train_ds,
    validation_data=val_ds,
    epochs=epochs_top,
    class_weight=class_weights,
    callbacks=[early_stop]
)
print("✅ Training completed.")

# ============================
# 10. Save Model as HDF5
# ============================
save_path = "./tf_model_1.keras"
model.save(save_path)   # No need for save_format='h5'
print(f"💾 Model saved to {save_path} ✅")



✅ TensorFlow session cleared.
📁 Dataset path: data
🖼 Image size: (128, 128), Batch size: 32
Found 150762 files belonging to 20 classes.
Using 120610 files for training.
Found 150762 files belonging to 20 classes.
Using 30152 files for validation.
📌 Classes detected (20): ['Acne And Rosacea Photos', 'Actinic Keratosis Basal Cell Carcinoma And Other Malignant Lesions', 'Atopic Dermatitis Photos', 'Ba  Cellulitis', 'Ba Impetigo', 'Benign', 'Bullous Disease Photos', 'Cellulitis Impetigo And Other Bacterial Infections', 'Eczema Photos', 'Exanthems And Drug Eruptions', 'Fu Athlete Foot', 'Fu Nail Fungus', 'Fu Ringworm', 'Hair Loss Photos Alopecia And Other Hair Diseases', 'Herpes Hpv And Other Stds Photos', 'Light Diseases And Disorders Of Pigmentation', 'Lupus And Other Connective Tissue Diseases', 'Malignant', 'Melanoma Skin Cancer Nevi And Moles', 'Rashes']
🎨 Data augmentation pipeline created.
✅ Training dataset preprocessed and cached in memory.
✅ Validation dataset preprocessed and cac

c:\Users\jayas\anaconda3\Lib\site-packages\keras\src\applications\mobilenet_v3.py:452: UserWarning: `input_shape` is undefined or non-square, or `rows` is not 224. Weights for input shape (224, 224) will be loaded as the default.
  return MobileNetV3(


🧱 MobileNetV3 Small model created (feature extraction).
⚙️ Model compiled.
⏹ EarlyStopping callback created.
🚀 Starting training (feature extraction)...
Epoch 1/20
3770/3770 ━━━━━━━━━━━━━━━━━━━━ 3571s 939ms/step - accuracy: 0.5312 - loss: 1.5058 - val_accuracy: 0.5691 - val_loss: 1.3470
Epoch 2/20
3770/3770 ━━━━━━━━━━━━━━━━━━━━ 2982s 790ms/step - accuracy: 0.5984 - loss: 1.2684 - val_accuracy: 0.5782 - val_loss: 1.3059
Epoch 3/20
3770/3770 ━━━━━━━━━━━━━━━━━━━━ 2317s 613ms/step - accuracy: 0.6123 - loss: 1.2173 - val_accuracy: 0.5797 - val_loss: 1.2993
Epoch 4/20
3770/3770 ━━━━━━━━━━━━━━━━━━━━ 2163s 573ms/step - accuracy: 0.6180 - loss: 1.1934 - val_accuracy: 0.5781 - val_loss: 1.3011
Epoch 5/20
3770/3770 ━━━━━━━━━━━━━━━━━━━━ 1919s 508ms/step - accuracy: 0.6213 - loss: 1.1799 - val_accuracy: 0.5767 - val_loss: 1.3054
Epoch 6/20
3770/3770 ━━━━━━━━━━━━━━━━━━━━ 2241s 593ms/step - accuracy: 0.6236 - loss: 1.1713 - val_accuracy: 0.5749 - val_loss: 1.3103
✅ Training completed.
💾 Model saved t

: 

In [2]:
# ============================
# 0. Imports & Clear Session
# ============================
import tensorflow as tf
import numpy as np
from sklearn.utils import class_weight
from tensorflow.keras.applications import MobileNetV3Small
from tensorflow.keras.applications.mobilenet_v3 import preprocess_input
from tensorflow.keras import backend as K
from tensorflow.keras.callbacks import EarlyStopping
import time  # for tracking training time

# Clear previous TF session
K.clear_session()
print("✅ TensorFlow session cleared.")

# ============================
# 1. Paths & Parameters
# ============================
data_dir = "data"
img_size = (128, 128)
batch_size = 32
epochs_top = 20
AUTOTUNE = tf.data.AUTOTUNE

print(f"📁 Dataset path: {data_dir}")
print(f"🖼 Image size: {img_size}, Batch size: {batch_size}")

# ============================
# 2. Load Dataset
# ============================
train_ds = tf.keras.utils.image_dataset_from_directory(
    data_dir,
    validation_split=0.2,
    subset="training",
    seed=123,
    image_size=img_size,
    batch_size=batch_size
)
val_ds = tf.keras.utils.image_dataset_from_directory(
    data_dir,
    validation_split=0.2,
    subset="validation",
    seed=123,
    image_size=img_size,
    batch_size=batch_size
)

class_names = train_ds.class_names
num_classes = len(class_names)
print(f"📌 Classes detected ({num_classes}): {class_names}")

# ============================
# 3. Data Augmentation
# ============================
data_augmentation = tf.keras.Sequential([
    tf.keras.layers.RandomFlip("horizontal"),
    tf.keras.layers.RandomRotation(0.2),
    tf.keras.layers.RandomZoom(0.2),
    tf.keras.layers.RandomContrast(0.15),
])
print("🎨 Data augmentation pipeline created.")

# ============================
# 4. Preprocessing
# ============================
def preprocess_train(x, y):
    x = tf.image.resize(x, img_size)
    x = data_augmentation(x, training=True)
    x = preprocess_input(x)
    return x, y

def preprocess_val(x, y):
    x = tf.image.resize(x, img_size)
    x = preprocess_input(x)
    return x, y

train_ds = train_ds.map(preprocess_train, num_parallel_calls=AUTOTUNE)
train_ds = train_ds.cache().prefetch(AUTOTUNE)
print("✅ Training dataset preprocessed and cached in memory.")

val_ds = val_ds.map(preprocess_val, num_parallel_calls=AUTOTUNE)
val_ds = val_ds.cache().prefetch(AUTOTUNE)
print("✅ Validation dataset preprocessed and cached in memory.")

# ============================
# 5. Class Weights
# ============================
y_train = np.concatenate([y for x, y in train_ds], axis=0)
class_weights = dict(enumerate(class_weight.compute_class_weight(
    'balanced', classes=np.unique(y_train), y=y_train
)))
print(f"⚖️ Class weights calculated: {class_weights}")

# ============================
# 6. Build Model (MobileNetV3 Small)
# ============================
base_model = MobileNetV3Small(
    input_shape=img_size + (3,),
    include_top=False,
    weights='imagenet'
)
base_model.trainable = False

x = tf.keras.layers.GlobalAveragePooling2D()(base_model.output)
output = tf.keras.layers.Dense(num_classes, activation='softmax', dtype='float32')(x)
model = tf.keras.Model(inputs=base_model.input, outputs=output)
print("🧱 MobileNetV3 Small model created (feature extraction).")

# ============================
# 7. Compile Model
# ============================
model.compile(
    optimizer=tf.keras.optimizers.Adam(learning_rate=0.001),
    loss='sparse_categorical_crossentropy',
    metrics=['accuracy']
)
print("⚙️ Model compiled.")

# ============================
# 8. Early Stopping Callback
# ============================
early_stop = EarlyStopping(
    monitor='val_accuracy',
    patience=3,
    restore_best_weights=True
)
print("⏹ EarlyStopping callback created.")

# ============================
# 9. Train Model with Time Tracking
# ============================
print("🚀 Starting training (feature extraction)...")
start_time = time.time()  # Start timer

history = model.fit(
    train_ds,
    validation_data=val_ds,
    epochs=epochs_top,
    class_weight=class_weights,
    callbacks=[early_stop]
)

end_time = time.time()  # End timer
elapsed_time = end_time - start_time
minutes = int(elapsed_time // 60)
seconds = int(elapsed_time % 60)
print(f"✅ Training completed in {minutes}m {seconds}s")

# ============================
# 10. Save Model in Best Format (.keras)
# ============================
save_path = "./tf_model.keras"
model.save(save_path)   # Saves in Keras V3 native format
print(f"💾 Model saved to {save_path} ✅")


✅ TensorFlow session cleared.
📁 Dataset path: data
🖼 Image size: (128, 128), Batch size: 32
Found 150762 files belonging to 20 classes.
Using 120610 files for training.
Found 150762 files belonging to 20 classes.
Using 30152 files for validation.
📌 Classes detected (20): ['Acne And Rosacea Photos', 'Actinic Keratosis Basal Cell Carcinoma And Other Malignant Lesions', 'Atopic Dermatitis Photos', 'Ba  Cellulitis', 'Ba Impetigo', 'Benign', 'Bullous Disease Photos', 'Cellulitis Impetigo And Other Bacterial Infections', 'Eczema Photos', 'Exanthems And Drug Eruptions', 'Fu Athlete Foot', 'Fu Nail Fungus', 'Fu Ringworm', 'Hair Loss Photos Alopecia And Other Hair Diseases', 'Herpes Hpv And Other Stds Photos', 'Light Diseases And Disorders Of Pigmentation', 'Lupus And Other Connective Tissue Diseases', 'Malignant', 'Melanoma Skin Cancer Nevi And Moles', 'Rashes']
🎨 Data augmentation pipeline created.
✅ Training dataset preprocessed and cached in memory.
✅ Validation dataset preprocessed and cac

c:\Users\jayas\anaconda3\Lib\site-packages\keras\src\applications\mobilenet_v3.py:452: UserWarning: `input_shape` is undefined or non-square, or `rows` is not 224. Weights for input shape (224, 224) will be loaded as the default.
  return MobileNetV3(


🧱 MobileNetV3 Small model created (feature extraction).
⚙️ Model compiled.
⏹ EarlyStopping callback created.
🚀 Starting training (feature extraction)...
Epoch 1/20
3770/3770 ━━━━━━━━━━━━━━━━━━━━ 3881s 1s/step - accuracy: 0.5297 - loss: 1.5027 - val_accuracy: 0.5709 - val_loss: 1.3325
Epoch 2/20
3770/3770 ━━━━━━━━━━━━━━━━━━━━ 2105s 556ms/step - accuracy: 0.5970 - loss: 1.2664 - val_accuracy: 0.5796 - val_loss: 1.2973
Epoch 3/20
3770/3770 ━━━━━━━━━━━━━━━━━━━━ 2319s 614ms/step - accuracy: 0.6114 - loss: 1.2159 - val_accuracy: 0.5806 - val_loss: 1.2943
Epoch 4/20
3770/3770 ━━━━━━━━━━━━━━━━━━━━ 1863s 493ms/step - accuracy: 0.6178 - loss: 1.1922 - val_accuracy: 0.5791 - val_loss: 1.2986
Epoch 5/20
3770/3770 ━━━━━━━━━━━━━━━━━━━━ 1614s 427ms/step - accuracy: 0.6216 - loss: 1.1788 - val_accuracy: 0.5779 - val_loss: 1.3047
Epoch 6/20
3770/3770 ━━━━━━━━━━━━━━━━━━━━ 1595s 422ms/step - accuracy: 0.6236 - loss: 1.1704 - val_accuracy: 0.5759 - val_loss: 1.3111
✅ Training completed in 223m 21s
💾 Model

In [ ]:
# ============================
# 0. Imports & Clear Session
# ============================
import tensorflow as tf
import numpy as np
import json
import time
from collections import Counter
from tensorflow.keras import backend as K
from sklearn.utils import class_weight
from tensorflow.keras.callbacks import EarlyStopping, ReduceLROnPlateau, ModelCheckpoint
from tensorflow.keras.applications import EfficientNetB0
from tensorflow.keras.applications.efficientnet import preprocess_input as effnet_preprocess

# Clear previous TF session
K.clear_session()
print("✅ TensorFlow session cleared.")

# ============================
# 1. Paths & Parameters
# ============================
data_dir = "data"
img_size = (224, 224)  # EfficientNetB0 pretrained size
batch_size = 32
epochs_feature = 10     # feature extraction phase
epochs_finetune = 20    # fine-tuning phase
AUTOTUNE = tf.data.AUTOTUNE
checkpoint_path = "./best_model.keras"
final_model_path = "./tf_model.keras"
history_path = "./training_history.json"

print(f"📁 Dataset path: {data_dir}")
print(f"🖼 Image size: {img_size}, Batch size: {batch_size}")

# ============================
# 2. Load Dataset
# ============================
train_ds = tf.keras.utils.image_dataset_from_directory(
    data_dir,
    validation_split=0.2,
    subset="training",
    seed=123,
    image_size=img_size,
    batch_size=batch_size,
)

val_ds = tf.keras.utils.image_dataset_from_directory(
    data_dir,
    validation_split=0.2,
    subset="validation",
    seed=123,
    image_size=img_size,
    batch_size=batch_size,
)

class_names = train_ds.class_names
num_classes = len(class_names)
print(f"📌 Classes detected ({num_classes}): {class_names}")

# ============================
# 3. Data Augmentation
# ============================
data_augmentation = tf.keras.Sequential([
    tf.keras.layers.RandomFlip("horizontal"),
    tf.keras.layers.RandomRotation(0.25),
    tf.keras.layers.RandomZoom(0.2),
    tf.keras.layers.RandomContrast(0.15),
    tf.keras.layers.RandomTranslation(0.1, 0.1),
])

print("🎨 Data augmentation pipeline created.")

# ============================
# 4. Preprocessing Pipelines
# ============================
def preprocess_train(x, y):
    x = tf.image.resize(x, img_size)
    x = data_augmentation(x, training=True)
    x = effnet_preprocess(x)
    return x, y

def preprocess_val(x, y):
    x = tf.image.resize(x, img_size)
    x = effnet_preprocess(x)
    return x, y

train_ds = (train_ds
            .map(preprocess_train, num_parallel_calls=AUTOTUNE)
            .cache()
            .prefetch(AUTOTUNE))
print("✅ Training dataset preprocessed and cached in memory.")

val_ds = (val_ds
          .map(preprocess_val, num_parallel_calls=AUTOTUNE)
          .cache()
          .prefetch(AUTOTUNE))
print("✅ Validation dataset preprocessed and cached in memory.")

# ============================
# 5. Class Weights (memory-safe)
# ============================
print("⚖️ Calculating class weights safely...")
label_counts = Counter()
for _, y in train_ds.unbatch():
    label_counts[int(y.numpy())] += 1

total = sum(label_counts.values())
class_weights = {cls: total / (len(label_counts) * count) for cls, count in label_counts.items()}
print(f"⚖️ Class weights calculated: {class_weights}")

# ============================
# 6. Build Model (EfficientNetB0)
# ============================
base_model = EfficientNetB0(
    include_top=False,
    weights="imagenet",
    input_shape=img_size + (3,)
)
base_model.trainable = False  # Phase 1: feature extraction

inputs = tf.keras.Input(shape=img_size + (3,))
x = inputs
x = base_model(x, training=False)
x = tf.keras.layers.GlobalAveragePooling2D()(x)
x = tf.keras.layers.Dropout(0.35)(x)
outputs = tf.keras.layers.Dense(num_classes, activation="softmax")(x)
model = tf.keras.Model(inputs, outputs)

print("🧱 EfficientNetB0 model created.")

# ============================
# 7. Compile (Phase 1)
# ============================
model.compile(
    optimizer=tf.keras.optimizers.Adam(learning_rate=1e-3),
    loss="sparse_categorical_crossentropy",
    metrics=[
        "accuracy",
        tf.keras.metrics.SparseTopKCategoricalAccuracy(k=3, name="top-3-acc")
    ],
)
print("⚙️ Model compiled for feature extraction.")

# ============================
# 8. Callbacks
# ============================
class TimeHistory(tf.keras.callbacks.Callback):
    def on_train_begin(self, logs=None):
        self.epoch_times = []
    def on_epoch_begin(self, epoch, logs=None):
        self._start = time.time()
    def on_epoch_end(self, epoch, logs=None):
        dur = time.time() - self._start
        self.epoch_times.append(dur)
        m, s = int(dur // 60), int(dur % 60)
        print(f"⏱ Epoch {epoch+1} duration: {m}m {s}s")

early_stop = EarlyStopping(monitor='val_accuracy', patience=4, restore_best_weights=True)
reduce_lr = ReduceLROnPlateau(monitor='val_loss', factor=0.5, patience=2, min_lr=1e-6, verbose=1)
checkpoint = ModelCheckpoint(checkpoint_path, save_best_only=True, monitor='val_accuracy', verbose=1)
time_cb = TimeHistory()

# ============================
# 9. Train (Phase 1: Feature Extraction)
# ============================
print("🚀 Starting Phase 1 training (feature extraction)...")
phase1_start = time.time()

history1 = model.fit(
    train_ds,
    validation_data=val_ds,
    epochs=epochs_feature,
    class_weight=class_weights,
    callbacks=[early_stop, reduce_lr, checkpoint, time_cb],
)

phase1_elapsed = time.time() - phase1_start
m1, s1 = int(phase1_elapsed // 60), int(phase1_elapsed % 60)
print(f"✅ Phase 1 completed in {m1}m {s1}s")

# Load best from phase 1
model = tf.keras.models.load_model(checkpoint_path)
print("🔁 Loaded best Phase 1 checkpoint.")

# ============================
# 10. Fine-Tuning Setup (Phase 2)
# ============================
base_model.trainable = True
fine_tune_at = max(0, len(base_model.layers) - 50)

for i, layer in enumerate(base_model.layers):
    layer.trainable = (i >= fine_tune_at)
    if isinstance(layer, tf.keras.layers.BatchNormalization):
        layer.trainable = False

model.compile(
    optimizer=tf.keras.optimizers.Adam(learning_rate=1e-5),
    loss="sparse_categorical_crossentropy",
    metrics=[
        "accuracy",
        tf.keras.metrics.SparseTopKCategoricalAccuracy(k=3, name="top-3-acc")
    ],
)
print(f"🛠 Fine-tuning from layer index {fine_tune_at}. Recompiled with LR=1e-5.")

# ============================
# 11. Train (Phase 2: Fine-Tuning)
# ============================
print("🚀 Starting Phase 2 training (fine-tuning)...")
phase2_start = time.time()

history2 = model.fit(
    train_ds,
    validation_data=val_ds,
    epochs=epochs_finetune,
    class_weight=class_weights,
    callbacks=[early_stop, reduce_lr, checkpoint, time_cb],
)

phase2_elapsed = time.time() - phase2_start
m2, s2 = int(phase2_elapsed // 60), int(phase2_elapsed % 60)
print(f"✅ Phase 2 completed in {m2}m {s2}s")

# Load best overall
model = tf.keras.models.load_model(checkpoint_path)
print("🏆 Loaded best checkpoint across both phases.")

# ============================
# 12. Save Final Model & History
# ============================
model.save(final_model_path)
print(f"💾 Final model saved to {final_model_path} ✅")

full_history = {}
for key in set(list(history1.history.keys()) + list(history2.history.keys())):
    full_history[key] = history1.history.get(key, []) + history2.history.get(key, [])

with open(history_path, "w") as f:
    json.dump(full_history, f)
print(f"📊 Training history saved to {history_path} ✅")

# ============================
# 13. Summary
# ============================
total_elapsed = phase1_elapsed + phase2_elapsed
mt, st = int(total_elapsed // 60), int(total_elapsed % 60)
print(f"⏱ Total training time (both phases): {mt}m {st}s")



✅ TensorFlow session cleared.
📁 Dataset path: data
🖼 Image size: (224, 224), Batch size: 32
Found 150762 files belonging to 20 classes.
Using 120610 files for training.
Found 150762 files belonging to 20 classes.
Using 30152 files for validation.
📌 Classes detected (20): ['Acne And Rosacea Photos', 'Actinic Keratosis Basal Cell Carcinoma And Other Malignant Lesions', 'Atopic Dermatitis Photos', 'Ba  Cellulitis', 'Ba Impetigo', 'Benign', 'Bullous Disease Photos', 'Cellulitis Impetigo And Other Bacterial Infections', 'Eczema Photos', 'Exanthems And Drug Eruptions', 'Fu Athlete Foot', 'Fu Nail Fungus', 'Fu Ringworm', 'Hair Loss Photos Alopecia And Other Hair Diseases', 'Herpes Hpv And Other Stds Photos', 'Light Diseases And Disorders Of Pigmentation', 'Lupus And Other Connective Tissue Diseases', 'Malignant', 'Melanoma Skin Cancer Nevi And Moles', 'Rashes']
🎨 Data augmentation pipeline created.
✅ Training dataset preprocessed and cached in memory.
✅ Validation dataset preprocessed and ca